In [50]:
import pandas as pd
import numpy as np
from datetime import datetime
#import pytz
import scipy
import requests
import warnings
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

# Read in Excel Data

In [2]:
excel_path = '/'.join(['.','Data',
                      'NBA Stats_2021-2023_01292024.xlsx'])

print('Reading in games...')
games = pd.read_excel(excel_path,
                     sheet_name = 'Games',
                     header = 0,
                     index_col = 'game_id',
                     parse_dates = ['game_date'],
                     engine='openpyxl')

print('Reading in players...')
players = pd.read_excel(excel_path,
                     sheet_name = 'Players',
                     header = 0,
                       index_col = 'player_id',
                       engine='openpyxl')

print('Reading in teams...')
teams = pd.read_excel(excel_path,
                     sheet_name = 'Teams',
                     header = 0,
                     index_col = 'team_id',
                     engine='openpyxl')

print('Reading in stats...')
stats = pd.read_excel(excel_path,
                     sheet_name = 'Stats',
                     header = 0,
                     engine='openpyxl')

Reading in games...
Reading in players...
Reading in teams...
Reading in stats...


In [3]:
games.head()

,game_date,game_season,game_status,game_period,game_time,game_home_team_id,game_home_team_score,game_visitor_team_id,game_visitor_team_score,game_preseason,game_postseason
game_id,,,,,,,,,,,
473410,2021-10-19,2021,Final,4,NaN,17,127,3,104,N,False
473409,2021-10-19,2021,Final,4,NaN,14,114,10,121,N,False
473413,2021-10-20,2021,Final,4,NaN,28,83,30,98,N,False
473415,2021-10-20,2021,Final,4,NaN,15,132,6,121,N,False
473414,2021-10-20,2021,Final,6,NaN,20,138,2,134,N,False


In [4]:
players.head()

,player_team_id,player_position,player_full_name,player_first_name,player_last_name,player_height_feet,player_height_inches,player_weight_pounds
player_id,,,,,,,,
17896049,3,G,David Duke Jr.,David,Duke Jr.,NaN,NaN,NaN
17553995,14,G,Austin Reaves,Austin,Reaves,NaN,NaN,NaN
666543,14,F,Sekou Doumbouya,Sekou,Doumbouya,NaN,NaN,NaN
59,14,G,Avery Bradley,Avery,Bradley,6.0,2.0,180.0
220,14,C,Dwight Howard,Dwight,Howard,6.0,11.0,265.0


In [5]:
stats.head()

,id,game_id,player_id,team_id,opponent_team_id,min,fga,fgm,fg_pct,fta,ftm,ft_pct,fg3a,fg3m,fg3_pct,oreb,dreb,reb,pts,ast,stl,blk,pf,turnover
0,7315088,473410,17896049,3,17,0:00,0,0,0.000,0,0,0.00,0,0,0.000,0,0,0,0,0,0,0,0,0
1,7315614,473409,17553995,14,10,0:00,0,0,0.000,0,0,0.00,0,0,0.000,0,0,0,0,0,0,0,0,0
2,7315611,473409,666543,14,10,0:00,0,0,0.000,0,0,0.00,0,0,0.000,0,0,0,0,0,0,0,0,0
3,7315610,473409,59,14,10,8,3,2,0.667,0,0,NaN,3,2,0.667,1,0,1,6,1,0,0,2,1
4,7315612,473409,220,14,10,13,2,1,0.500,4,3,0.75,0,0,NaN,0,6,6,5,0,0,0,2,2


In [6]:
# Convert 'min' to numeric and fill NaN with 0
stats['min'] = pd.to_numeric(stats['min'],
                             errors = 'coerce')\
                .fillna(0)

In [7]:
# Fantasy points (PrizePicks)
## Points = 1
## Rebound = 1.2
## Assists = 1.5
## Block = 3
## Steals = 3
## Turnover = -1
stats['fpts'] = stats['pts']\
                + (1.2 * stats['reb'])\
                + (1.5 * stats['ast'])\
                + (3 * stats['blk'])\
                + (3 * stats['stl'])\
                + (-1 * stats['turnover'])

In [8]:
# Merge date of games
stats_dates = pd.merge(stats,
                      games[['game_date']],
                      how = 'left',
                      left_on = 'game_id',
                      right_index = True)\
                .sort_values(['game_date','player_id'])

In [9]:
# Merge player positions
stats_dates_pos = pd.merge(stats_dates,
                           players[['player_position']],
                           how = 'left',
                           left_on = 'player_id',
                           right_index = True)

In [10]:
# Calculate pts per 36 min by game
#stats_dates_pos['pts_per36'] = 36 * (stats_dates_pos['pts']/stats_dates_pos['min'])

In [11]:
# Determine rolling_period on rolling metrics for entire analysis
rolling_period = 62    # Based on number of observations/games, not number of days

# rolling_period_days = '120D'    # Based on number of days, not number of observations/games

# Team Stats

In [12]:
stats_dates_pos.head()

,id,game_id,player_id,team_id,opponent_team_id,min,fga,fgm,fg_pct,fta,ftm,ft_pct,fg3a,fg3m,fg3_pct,oreb,dreb,reb,pts,ast,stl,blk,pf,turnover,fpts,game_date,player_position
26,7315084,473410,6,3,17,16.0,2,0,0.000,4,1,0.250,0,0,NaN,0,3,3,1,1,0,1,2,1,8.1,2021-10-19,F
35,7315097,473410,8,17,3,28.0,10,3,0.300,1,1,1.000,10,3,0.30,0,4,4,10,6,2,0,0,0,29.8,2021-10-19,G
48,7315095,473410,15,17,3,31.0,25,12,0.480,9,7,0.778,4,1,0.25,5,9,14,32,7,1,2,4,4,64.3,2021-10-19,F
6,7315609,473409,17,14,10,26.0,9,3,0.333,2,1,0.500,4,2,0.50,0,4,4,9,2,0,0,2,1,15.8,2021-10-19,F
9,7315607,473409,36,14,10,31.0,9,3,0.333,0,0,NaN,8,2,0.25,0,2,2,8,0,0,1,4,1,12.4,2021-10-19,G


In [13]:
def team_pts_rolling_mean(group):
    group_indexed = group.set_index('game_date')
    
    rolling_group = group_indexed\
                    [['pts']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .mean()\
                    .rename(columns = {'pts':'ppg'})
    
    return rolling_group

## Defensive Efficiency by Position

In [14]:
team_def_eff_pos = stats_dates_pos\
                        .groupby(['opponent_team_id','game_date','player_position'])\
                        [['pts']]\
                        .sum()\
                        .reset_index()

In [15]:
team_stats_rolling_def_eff = team_def_eff_pos.groupby(['opponent_team_id','player_position'])\
                            .apply(team_pts_rolling_mean)\
                            .reset_index()\
                            .rename(columns = {'opponent_team_id':'team_id',
                                              'ppg':'def_ppg'})

## Offensive Efficiency by Position

In [16]:
team_off_eff_pos = stats_dates_pos\
                        .groupby(['team_id','game_date','player_position'])\
                        [['pts']]\
                        .sum()\
                        .reset_index()

In [17]:
team_stats_rolling_off_eff = team_off_eff_pos.groupby(['team_id','player_position'])\
                            .apply(team_pts_rolling_mean)\
                            .reset_index()\
                            .rename(columns = {'ppg':'off_ppg'})

## Merge Team Defensive and Offensive Efficiency by Position

In [18]:
team_eff_pos = pd.merge(team_stats_rolling_off_eff,
                        team_stats_rolling_def_eff,
                        how = 'outer',
                        left_on = ['team_id','game_date','player_position'],
                        right_on = ['team_id','game_date','player_position'])

## League Team Averages

In [19]:
def league_team_pts_stats(group):
    '''
    Grouping function to get standard deviation of pts_per36 for each player on a given date
    Need to account for offseason/days where there are no games
    '''
    # Sort by date and player prior to resampling
    group_sorted = group.sort_values(['game_date','team_id'])\
                        .set_index(['game_date'])
    
    # Resample data to daily by each player
    ## Forward fill blank values
    group_resampled = group_sorted.groupby('team_id')\
                        .resample('1D')\
                        [['pts_off','pts_def']]\
                        .last()
    
    # Were games played on date
    ## Due to resample, dates in the off season were added
    ## Need to remove; will cause calculations over at the beginning of each season
    date_no_minutes = group_resampled.groupby(level = 1)\
                        .apply(lambda x: x.isna()\
                                           .all()
                              )
    
    # Drop dates with no games
    ## Includes in-season breaks
    date_no_games = date_no_minutes[(date_no_minutes['pts_off'] == True)
                                   & (date_no_minutes['pts_def'] == True)].index
    
    group_resampled.drop(index = date_no_games,
                        level = 1,
                        inplace = True)
    
    # Forward fill values by player
    final_group_resampled = group_resampled.groupby(level = [0])[['pts_off','pts_def']].ffill()
    
    # Calculate the standard deviation of pts_per36 for all players by date
    final_rolling_stats = final_group_resampled.groupby(level = 1).agg(['mean',
                                                                        'std'])
    
    return final_rolling_stats

In [20]:
# Calculate teams offensive pts production by position per game
team_pos_off_pts_total = stats_dates_pos.groupby(['game_date',
                                                  'player_position',
                                                  'team_id'])\
                            [['pts']]\
                            .sum()\
                            .reset_index()\
                            .rename(columns = {'pts':'pts_off'})

# Calculate teams defensive pts production by position per game
team_pos_def_pts_total = stats_dates_pos.groupby(['game_date',
                                                  'player_position',
                                                  'opponent_team_id'])\
                            [['pts']]\
                            .sum()\
                            .reset_index()\
                            .rename(columns = {'opponent_team_id':'team_id',
                                               'pts':'pts_def'})

# Merge teams offensive and defensive production by position
team_pos_eff_total = pd.merge(team_pos_off_pts_total,
                               team_pos_def_pts_total,
                               how = 'outer',
                               left_on = ['game_date','team_id','player_position'],
                               right_on = ['game_date','team_id','player_position'])

In [21]:
team_pos_rolling_eff = team_pos_eff_total\
                        .groupby('player_position')\
                        .apply(league_team_pts_stats)

In [22]:
team_pos_rolling_eff.columns = ['_'.join(i) for i in team_pos_rolling_eff.columns]

In [23]:
team_pos_rolling_eff.reset_index(inplace = True)

# Player Stats

## Individual Player Efficiency

In [24]:
def player_pts_rolling_sum(group):
    group_indexed = group.sort_values('game_date')\
                    .set_index('game_date')
    
    rolling_group = group_indexed\
                        [['min','pts']]\
                       .rolling(rolling_period,
                                min_periods = 1)\
                       .sum()
    
    rolling_min_mean = group_indexed\
                        [['min']]\
                        .rolling(rolling_period,
                                 min_periods = 1)\
                        .mean()
    
    rolling_merged = pd.merge(rolling_min_mean,
                              rolling_group,
                              how = 'outer',
                              left_index = True,
                              right_index = True,
                             suffixes = ['_mean','_cumsum'])
    
    return rolling_merged

In [25]:
player_stats_rolling_eff = stats_dates.groupby('player_id')\
                            .apply(player_pts_rolling_sum)

# player_stats_rolling_eff = stats_dates.set_index('game_date')\
#                             .groupby(stats_dates['player_id'])\
#                             [['min','pts']]\
#                             .rolling(rolling_period,
#                                     min_periods = 1)\
#                             .sum()\
#                             .shift(1)\
#                             .dropna(how = 'all')

In [26]:
player_stats_rolling_eff

min_mean  min_cumsum   pts
player_id game_date                             
3         2021-10-20     32.00        32.0   8.0
          2021-10-23     29.50        59.0  25.0
          2021-10-24     31.00        93.0  39.0
          2021-10-27     27.75       111.0  43.0
          2021-10-28     26.40       132.0  55.0
...                        ...         ...   ...
59421201  2023-12-13      0.00         0.0   0.0
          2023-12-15      0.00         0.0   0.0
          2023-12-16      0.00         0.0   0.0
          2023-12-18      0.00         0.0   0.0
          2023-12-20      0.00         0.0   0.0

[96937 rows x 3 columns]

In [27]:
stat_categories = player_stats_rolling_eff.columns.drop(['min_mean','min_cumsum'])

In [28]:
for cat in stat_categories:
    player_stats_rolling_eff[cat+'_per36'] = player_stats_rolling_eff[cat] * (36/player_stats_rolling_eff['min_cumsum'])

In [29]:
actual_eff_merged = pd.merge(stats_dates_pos[['game_date','player_id','player_position',
                                              'min','pts']],
                             player_stats_rolling_eff.reset_index()\
                                 [['game_date','player_id','pts_per36','min_mean']],
                             how = 'left',
                             left_on = ['player_id','game_date'],
                             right_on = ['player_id','game_date'])#\
                    #.sort_values(['player_id','game_date'])

In [30]:
per36_stat_cols = [i for i in actual_eff_merged.columns if ('_per36' in i)]

## League Position Average

In [31]:
def league_player_pts_stats(group):
    '''
    Grouping function to get standard deviation of pts_per36 for each player on a given date
    Need to account for offseason/days where there are no games
    '''
    # Sort by date and player prior to resampling
    group_sorted = group.sort_values(['game_date','player_id'])\
                        .set_index(['game_date'])
    
    # Resample data to daily by each player
    ## Forward fill blank values
    group_resampled = group_sorted.groupby(['player_id',
                                           ])\
                        .resample('1D')\
                        [['pts_per36']]\
                        .last()
    
    # Were games played on date
    ## Due to resample, dates in the off season were added
    ## Need to remove; will cause calculations over at the beginning of each season
    date_no_minutes = group_resampled.groupby(level = 1)\
                        .apply(lambda x: x.isna()\
                                           .all()
                              )
    
    # Drop dates with no games
    ## Includes in-season breaks
    date_no_games = date_no_minutes[(date_no_minutes['pts_per36'] == True)].index
    
    group_resampled.drop(index = date_no_games,
                        level = 1,
                        inplace = True)
    
    # Forward fill values by player
    final_group_resampled = group_resampled.groupby(level = [0])[['pts_per36']].ffill()
    
    # Calculate the standard deviation of pts_per36 for all players by date
    final_rolling_stats = final_group_resampled.groupby(level = [1]).agg(['mean',
                                                                        'std'])
    
    return final_rolling_stats

In [32]:
league_player_rolling_stats = actual_eff_merged.groupby('player_position',
                                                       dropna = False)\
                                .apply(league_player_pts_stats)

In [33]:
league_player_rolling_stats.columns = ['_'.join(i) for i in league_player_rolling_stats.columns]

In [34]:
league_player_rolling_stats.reset_index(inplace = True)

# Shift Data
Need to shift team efficiency, player efficiency, and league rolling data down a row so they can be used as dependent variables on the actual predictor variable

In [35]:
team_eff_pos_shifted = team_eff_pos.groupby(['team_id','player_position'])\
                        .apply(lambda x: x.sort_values('game_date')\
                                           .set_index('game_date')\
                                           [['off_ppg','def_ppg']]\
                                           .shift(1))\
                        .reset_index()

In [36]:
team_pos_rolling_eff_shifted = team_pos_rolling_eff.groupby('player_position')\
                                .apply(lambda x: x.sort_values('game_date')\
                                                   .set_index('game_date')\
                                                   [['pts_off_mean','pts_off_std',
                                                     'pts_def_mean','pts_def_std']]\
                                                   .shift(1))\
                                .reset_index()

In [37]:
actual_eff_merged_shifted = actual_eff_merged.groupby(['player_id','player_position'])\
                                .apply(lambda x: x.sort_values('game_date')\
                                                   .set_index('game_date')\
                                                   [['pts_per36','min_mean']]\
                                                   .shift(1))\
                                .reset_index()

In [38]:
league_player_rolling_stats_shifted = league_player_rolling_stats.groupby('player_position')\
                                        .apply(lambda x: x.sort_values('game_date')\
                                                           .set_index('game_date')\
                                                           [['pts_per36_mean','pts_per36_std']]\
                                                           .shift(1))\
                                        .reset_index()

# Merge Data

## Merge Team Stats

In [39]:
rolling_stats_teams = pd.merge(team_eff_pos_shifted.rename(columns = {'off_ppg':'team_off_ppg',
                                                                      'def_ppg':'team_def_ppg'}),
                               team_pos_rolling_eff_shifted.rename(columns = {'pts_off_mean':'league_ppg_off_mean',
                                                                              'pts_off_std':'league_ppg_off_std',
                                                                              'pts_def_mean':'league_ppg_def_mean',
                                                                              'pts_def_std':'league_ppg_def_std'}),
                               how = 'outer',
                               left_on = ['player_position','game_date'],
                               right_on = ['player_position','game_date'])\
                        .sort_values(['game_date','player_position','team_id'])

## Merge Player Stats

In [40]:
rolling_stats_players = pd.merge(actual_eff_merged_shifted.rename(columns = {'pts_per36':'player_pts_per36',
                                                                            'min_mean':'player_avg_min'}),
                                 league_player_rolling_stats_shifted.rename(columns = {'pts_per36_mean':'league_player_pts_per36_mean',
                                                                                       'pts_per36_std':'league_player_pts_per36_std'}),
                                 how = 'outer',
                                 left_on = ['player_position','game_date'],
                                 right_on = ['player_position','game_date'])\
                        .sort_values(['game_date','player_position','player_id'])

## Merge Player and Team Stats to Relevant Stats

In [41]:
# Table of relevant fields from original data
rel_stats = stats_dates_pos[['game_date',
                             'player_id','player_position',
                             'team_id','opponent_team_id',
                             'min','pts']]

In [42]:
rel_rolling_players = pd.merge(rel_stats,
                               rolling_stats_players,
                               how = 'outer',
                               left_on = ['game_date','player_id','player_position'],
                               right_on = ['game_date','player_id','player_position'])
                         

In [43]:
rel_rolling_team_off = pd.merge(rel_rolling_players,
                                rolling_stats_teams[['team_id','player_position','game_date',
                                                     'team_off_ppg','league_ppg_off_mean','league_ppg_off_std'
                                                    ]],
                                how = 'left',
                                left_on = ['game_date','team_id','player_position'],
                                right_on = ['game_date','team_id','player_position'])

In [44]:
rel_rolling_team_opp = pd.merge(rel_rolling_team_off,
                                rolling_stats_teams[['team_id','player_position','game_date',
                                                     'team_def_ppg','league_ppg_def_mean','league_ppg_def_std'
                                                    ]]\
                                    .rename(columns = {'team_def_ppg':'opponent_team_def_ppg'}),
                                how = 'left',
                                left_on = ['game_date','opponent_team_id','player_position'],
                                right_on = ['game_date','team_id','player_position'],
                               suffixes = ['','_y'])\
                        .drop('team_id_y',
                              axis = 1)

# Standardize Data

## Team Data Standardization

In [45]:
rel_rolling_team_opp['team_off_ppg_stand'] = (rel_rolling_team_opp['team_off_ppg'] - rel_rolling_team_opp['league_ppg_off_mean'])/rel_rolling_team_opp['league_ppg_off_std']

## Opponent Team Data Standardization

In [46]:
rel_rolling_team_opp['opp_def_ppg_stand'] = (rel_rolling_team_opp['opponent_team_def_ppg'] - rel_rolling_team_opp['league_ppg_def_mean'])/rel_rolling_team_opp['league_ppg_def_std']

## Player Data Standardization

In [47]:
rel_rolling_team_opp['player_pts_per36_stand'] = (rel_rolling_team_opp['player_pts_per36'] - rel_rolling_team_opp['league_player_pts_per36_mean'])/rel_rolling_team_opp['league_player_pts_per36_std']

In [48]:
rel_rolling_team_opp[rel_rolling_team_opp['player_id'] == 175]

,game_date,player_id,player_position,team_id,opponent_team_id,min,pts,player_pts_per36,player_avg_min,league_player_pts_per36_mean,league_player_pts_per36_std,team_off_ppg,league_ppg_off_mean,league_ppg_off_std,opponent_team_def_ppg,league_ppg_def_mean,league_ppg_def_std,team_off_ppg_stand,opp_def_ppg_stand,player_pts_per36_stand
102,2021-10-20,175,G,21,29,33.0,18,NaN,NaN,17.554420,8.498880,NaN,44.500000,11.090537,NaN,44.500000,11.090537,NaN,NaN,NaN
500,2021-10-22,175,G,21,11,26.0,13,19.636364,33.000000,17.520714,8.477844,42.000000,48.366667,17.223647,74.000000,49.366667,16.959350,-0.369647,1.452493,0.249550
993,2021-10-24,175,G,21,23,39.0,29,18.915254,29.500000,17.013971,7.618298,42.000000,44.800000,15.831875,42.500000,46.066667,14.330901,-0.176858,-0.248879,0.249568
1409,2021-10-26,175,G,21,10,36.0,30,22.040816,32.666667,17.062895,6.751355,49.000000,48.333333,15.660918,46.666667,49.166667,16.681775,0.042569,-0.149864,0.737322
1576,2021-10-27,175,G,21,14,38.0,27,24.179104,33.500000,17.199990,6.997151,50.000000,48.700000,16.295017,58.750000,48.466667,16.627218,0.079779,0.618464,0.997422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95200,2024-01-23,175,G,21,25,35.0,33,32.684458,30.822581,15.959434,6.383956,75.741935,46.266667,17.896124,52.419355,46.833333,16.844283,1.647020,0.331627,2.619853
95389,2024-01-24,175,G,21,27,29.0,32,32.707091,31.387097,15.948809,6.388545,75.500000,45.966667,17.564626,53.193548,46.700000,16.563670,1.681410,0.392036,2.623177
95897,2024-01-26,175,G,21,19,31.0,31,32.730650,31.258065,15.847028,5.818666,75.806452,49.700000,20.457104,51.967742,51.133333,20.992172,1.276156,0.039749,2.901631
96516,2024-01-28,175,G,21,9,29.0,31,32.778065,31.177419,15.718794,5.663501,75.387097,45.500000,25.243299,50.709677,40.733333,22.715532,1.183962,0.439186,3.012142


In [72]:
player_id = 37

fig = plt.figure(figsize = [30,10])
ax = sns.pointplot(data = rel_rolling_team_opp[rel_rolling_team_opp['player_id'] == player_id],
                 x = 'game_date',
                 y = 'player_pts_per36_stand',
                ci = None # Confidence Interval
                )
ax.set_title(players[players.index == player_id]['player_full_name'][player_id])
ax.set_xlabel('Game Date')
ax.set_ylabel('Player Points per 36min Standard Dev')

xaxis_date_format = mdates.DateFormatter('%y-%m')
plt.gca().xaxis.set_major_formatter(xaxis_date_format)

plt.xticks(rotation=90)
# fig.tight_layout()
fig.show()

In [139]:
players[players['player_last_name'] == 'Beal']

,player_team_id,player_position,player_full_name,player_first_name,player_last_name,player_height_feet,player_height_inches,player_weight_pounds
player_id,,,,,,,,
37,24,G,Bradley Beal,Bradley,Beal,6.0,5.0,207.0


In [140]:
teams[teams.index == 24]

,team_full_name,team_name,team_abbreviation,team_city,team_conference,team_division
team_id,,,,,,
24,Phoenix Suns,Suns,PHX,Phoenix,West,Pacific


In [73]:
excel_export_path = '/'.join(['.','Data',
                              'NBA Points Stats_2021-2023.xlsx'])

rel_rolling_team_opp.to_excel(excel_export_path,
                              sheet_name = 'Points',
                              index = False)

# Extra

In [ ]:
def league_team_pts_rolling_mean(group):
    group_indexed = group.set_index('game_date')
    
    rolling_group = group_indexed.groupby(['opponent_team_id','player_position'])\
                    [['pts']]\
                    .rolling(rolling_period,
                             min_periods = 1)\
                    .mean()\
                    .shift(1)\
                    .rename(columns = {'pts':'ppg'})

    final_rolling = rolling_group.droplevel([0,1])
    
    return final_rolling

In [ ]:
team_stats_rolling_eff[(team_stats_rolling_eff['opponent_team_id'] == 1)
                       & (team_stats_rolling_eff['player_position'] == 'C')]

In [ ]:
team_stats_by_game[team_stats_by_game['team_id'] == 10]